<a href="https://albertomerani.org/">
    <img src="https://drive.google.com/uc?export=view&id=16uZ9pqgmRVKrXRhDO_acSUe0ZJAfuy1C" width="1000" align="center">
</a>

# **CASO DE NEGOCIO: Predicción de vivienda**


<h2>Tabla de contenidos</h2>
<div class="alerta alerta-bloque alerta-info" estilo="margin-top: 20px">
     <ul>
         <ul>
             <li><a href="#sin"> Sinopsis </a></li>
             <li><a href="#pp"> Principales Pasos </a></li>
             <ul>             
                 <li><a href="#neg"> Paso 1: Entendimiento de negocio </a></li>
                 <li><a href="#dat"> Paso 2: Entendimiento de los datos </a></li>
                 <li><a href="#prep"> Paso 3: Preparación de Datos</a></li>
                 <li><a href="#mod"> Paso 4: Modelamiento</a></li>
                 <li><a href="#eva"> Paso 5: Evalucación</a></li>
            </ul>
         </ul>
     </ul>
</div>

<hr>

<h2 id="sin"> Sinopsis </h2>

"Este conjunto de datos es una versión modificada del conjunto de datos de Vivienda de California disponible en la página de Luís Torgo (Universidad de Oporto). Luís Torgo lo obtuvo del repositorio de StatLib (que ahora está cerrado). El conjunto de datos también se puede descargar desde los espejos de StatLib.

La siguiente es la descripción del autor del libro:

Este conjunto de datos apareció en un artículo de 1997 titulado Sparse Spatial Autoregressions de Pace, R. Kelley y Ronald Barry, publicado en la revista Statistics and Probability Letters. Lo construyeron utilizando los datos del censo de California de 1990. Contiene una fila por grupo de bloques censales. Un grupo de bloques es la unidad geográfica más pequeña para la cual la Oficina del Censo de EE. UU. publica datos de muestra (un grupo de bloques suele tener una población de 600 a 3000 personas).

El conjunto de datos de este directorio es casi idéntico al original, con dos diferencias:
Se eliminaron aleatoriamente 207 valores de la columna total_bedrooms, por lo que podemos discutir qué hacer con los datos faltantes.
Se agregó un atributo categórico adicional llamado ocean_proximity, que indica (de manera muy aproximada) si cada grupo de bloques está cerca del océano, cerca del área de la Bahía, tierra adentro o en una isla. Esto permite discutir qué hacer con los datos categóricos.

 <h2 id="pp"> Principales Pasos en un Proyecto de I.A. </h2>

1. Entendimiento de negocio
2. Entendimiento de los datos.
    * tipos de datos (número, texto, objeto, etc.)
    * continuo/discreto
    * estadísticas básicas (mín., máx., estándar, mediana, etc.) mediante diagrama de caja
    * frecuencia a través del histograma
    * escalas y distribuciones de diferentes características
    * Análisis de correlación (combinaciones por pares y atributos)
3. Prepraración de Datos.
    * Limpieza de datos (datos faltantes, valores atípicos, errores de datos
    * Transformación de datos a través de canalizaciones (texto categórico a número usando una codificación activa, escalado de características mediante normalización/estandarización, combinaciones de características)
4. Modelamiento
    * Entrene y valide de forma cruzada diferentes modelos y seleccione el más prometedor
    * Ajuste el modelo probando diferentes combinaciones de hiperparámetros.
5. Evaluación.
6. Iniciar, monitorear y actualizar el modelo y el sistema.et *(Este paso no se llevar a cabo por el alcanze del curso)*

<h2 id="neg"> Paso 1: Entendimiento de negocio </h2>

Este problema de clasificación consiste en estimar la ubicación aproximada de los bloques de viviendas. La ubicación aproximada se representa con una variable discreta llamada ocean_proximity que puede tener uno de los cinco valores posibles:

* **NEAR BAY**  -  CERCA DE LA BAHÍA
* **<1H OCEAN** -  <1H OCÉANO
* **INLAND**    -  INTERIOR
* **NEAR OCEAN** - CERCA DEL OCÉANO
* **ISLAND**   -   ISLA

<h3 id="neg"> Contenido </h3>

Los datos pertenecen a las casas encontradas en un distrito determinado de California y algunas estadísticas resumidas sobre ellas basadas en los datos del censo de 1990. ¡Tenga en cuenta que los datos no se limpian, por lo que se requieren algunos pasos de preprocesamiento! Las columnas son las siguientes, sus nombres se explican por sí solos:

* longitude
* latitude
* housing_median_age
* total_rooms
* population
* households
* median_income
* median_house_value
* ocean_proximity

<h2 id="dat"> Paso 2: Entendimiento de los datos </h2>

In [ ]:
# Establezca el número máximo de subprocesos en 4 en el computador, para un cálculo más rápido
import os
os.environ["OMP_NUM_THREADS"] = "4"

### **Cargar los paquetes y librerías necesarias**

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from math import sqrt

# data viz
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(color_codes=True)
sns.set_palette(sns.color_palette("muted"))


import warnings
warnings.filterwarnings("ignore")

### **Cargar los datos**

Verificar el **directorio** de los archivos para que se pueda leer la base de datos

In [ ]:
ls /content/drive/MyDrive/MINTIC/

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/MINTIC/')

In [ ]:
Housing_df = pd.read_csv("data/HousingRawDataset.csv", encoding='utf-8')
Housing_df

### **Análisis exploratorio de datos - EDA**

En ciencia de datos aplicamos el método científico a los datos con el objetivo de obtener conocimientos. Esto significa que planteamos una hipótesis sobre los datos, la probamos y la refinamos si es necesario. En este marco, el análisis exploratorio de datos (EDA) es el paso en el que exploramos los datos antes de construir modelos. Esto nos ayuda a comprender qué información contienen realmente los datos y qué conocimientos se pueden obtener de ellos.

Formalmente, los objetivos de EDA son:

* Sugerir hipótesis sobre los fenómenos de interés.
* Compruebar si hay datos necesarios disponibles para probar estas hipótesis.
* Hacer una selección de métodos y modelos adecuados para lograr el objetivo.
* Sugerir qué datos deberían recopilarse para una mayor investigación.

Esta fase exploratoria marca el camino para el resto de un proyecto de ciencia de datos y, por lo tanto, es una parte crucial del proceso.

In [ ]:
# ver la información básica
Housing_df.info()

**¿Qué información debemos obtener de info()?**

* Total de observaciones: 20640: asegúrese de comprender qué es cada observación. En este caso, cada observación son los datos de un registro.
* Columnas totales (Variables): 10
* Tipo de datos de cada característica: 9 númericos y 1 objeto (se manejará más adelante)
* Significado de cada característica: es muy importante trabajar con un experto en el dominio para comprender completamente cada característica
* Valores nulos (por ejemplo, total_bedrooms es 20433, se manejará más adelante)

In [ ]:
Housing_df.head()

### **Data Types**

head() muestra el valor de las filas superiores, lo que da más idea sobre los tipos de datos. Pandas adivinó los tipos de datos de las características al leer los datos, lo que puede no siempre funcionar. En este conjunto de datos, puede ver que la característica ocean_proximity es texto. A veces, la característica de precio también puede ser de tipo **objeto** porque los datos sin procesar tienen el signo **$**, en cuyo caso necesita convertir el tipo de datos de objeto a **float64** si desea utilizar esta característica en el modelo.

### **Estadísticas básicas**

describe() muestra un resumen de características numéricas, que se pueden visualizar mediante diagramas de caja e histogramas. value_counts() se puede utilizar para generar un resumen de características categóricas.

In [ ]:
Housing_df.describe()

In [ ]:
Housing_df["ocean_proximity"].value_counts()

In [ ]:
# Configura el contexto de Seaborn para "paper" y aumenta el tamaño de la fuente
# y el tamaño de la letra de los títulos y etiquetas de los ejes
sns.set_context("paper", rc={"font.size":20,"axes.titlesize":16,"axes.labelsize":11})

# Configura el tema o diseño de Seaborn a "white"
sns.set_theme(style="white")

# Crea un diagrama de dispersión de la columna "ocean_proximity" del conjunto de datos Housing_df
# con el color #f2AB6D, sin KDE, una altura de 6 pulgadas y 15 bins : INTETE CAMBIAR KDE=True
# Establece el título, las etiquetas de los ejes y las leyendas del diagrama
sns.displot(Housing_df['ocean_proximity'], color='#f2AB6D', kde=False, height=6, bins=15).set(
    title='Distribución Ocean Proximity', xlabel='', ylabel='')

### **Visualización de datos**

Otra forma rápida de tener una idea del tipo de datos numéricos que está tratando es trazar uno o más de los siguientes:

* **Histogramas**: muestra el número de instancias (en el eje vertical) que tienen un rango de valores determinado (en el eje horizontal). Útil para comprender la forma de una sola variable.
* **Mapa de calor de matriz de correlación**: muestra cuánto se correlaciona cada columna entre sí con un degradado de color. Útil para ver rápidamente qué variables se correlacionan más fuertemente con la variable de interés.
* **Gráficos de dispersión**: muestra una colección de puntos, cada uno de los cuales tiene el valor de una columna que determina la posición en el eje horizontal y el valor de la otra columna que determina la posición en el eje vertical. Útil para buscar correlaciones visualmente .

In [ ]:
# Crea un histograma de cada columna de Housing_df
# con 50 bins y un tamaño de figura de 20x15 pulgadas
Housing_df.hist(bins=50, figsize=(20,15))

# Muestra el histograma
plt.show()

#### **Mapa de calor de matriz de correlación**

Para crear el mapa de calor, primero debemos calcular la matriz de correlación de nuestras columnas numéricas.

<a href="https://en.wikipedia.org/wiki/Correlation">
    <img src="https://drive.google.com/uc?export=view&id=1h5uPxmiMsG7DcrRGrWvgwlcQzdszDIE8" width="700" align="center">
</a>


In [ ]:
# Esta línea de código crea una nueva figura de Matplotlib con un tamaño de 9x6 pulgadas.
plt.figure(figsize=(9,6))

# Crea una matriz triangular superior de la matriz de correlación de Housing_df
upp_mat = np.triu(Housing_df.corr(numeric_only=True))

# Selecciona solo las columnas numéricas
numeric_df = Housing_df.select_dtypes(include=[np.number])

# Calcula la matriz de correlación
corr_matrix = numeric_df.corr()

# Crea un mapa de calor de la matriz de correlación de Housing_df, con los siguientes argumentos:
# * vmin = -1: valor mínimo de la matriz de correlación
# * vmax = 1: valor máximo de la matriz de correlación
# * annot = True: muestra los valores de correlación en el mapa de calor
# * cmap = 'coolwarm': usa la paleta de colores 'coolwarm', INTENTE CAMBIAR AL 'viridis'
# * mask = upp_mat: enmascara el mapa de calor con la matriz triangular superior
sns.heatmap(corr_matrix, vmin = -1, vmax = +1, annot = True, cmap = 'coolwarm', mask = upp_mat)

Las variables **total rooms** esta correlacionada con las variable **population** y **households**, de igual manera **median_income** y **median_house_value** tambien tienden a tener una alta correlacion

#### **Gráfico de dispersión**

Finalmente, otra forma de comprobar las correlaciones es examinar los diagramas de dispersión de cada columna numérica. Esto puede resultar útil para detectar correlaciones no lineales que podrían pasarse por alto en el análisis anterior. Seaborn proporciona una función útil seaborn.pairplotque nos permite ver rápidamente las relaciones entre los datos numéricos:

In [ ]:
# Crea una lista de variables que se utilizarán en el diagrama de pares
attributes = ["median_house_value", "median_income", "total_rooms", "housing_median_age"]

# Crea un diagrama de pares de las columnas especificadas en la lista de variables, eliminando los valores nulos con dropna()
sns.pairplot(Housing_df[attributes].dropna());

In [ ]:
sns.set_context("paper", rc={"font.size":20,"axes.titlesize":16,"axes.labelsize":13})
sns.set_theme(style="white")

sns.displot(Housing_df['median_income'], color='#02AB6D', kde=False, height=5, bins=15).set(
    title='Distribución Median Income', xlabel='', ylabel='')

In [ ]:
sns.set_context("paper", rc={"font.size":20,"axes.titlesize":16,"axes.labelsize":13})
sns.set_theme(style="white")

sns.displot(Housing_df['median_house_value'], color='#020B6D', kde=False, height=5, bins=15).set(
    title='Distribución Median House Value', xlabel='', ylabel='')

#### **Boxplot**

Un gráfico boxplot, también conocido como diagrama de caja y bigotes, es un tipo de gráfico que se utiliza para representar gráficamente la distribución de una variable numérica. El gráfico está compuesto por seis elementos:

* Límite inferior: El valor más pequeño de la muestra.
* Primer cuartil (Q1): El valor que deja por debajo el 25% de los datos.
* Mediana: El valor que deja por debajo y por encima el 50% de los datos.
* Tercer cuartil (Q3): El valor que deja por encima el 75% de los datos.
* Límite superior: El valor más grande de la muestra.
* Bigotes: Líneas que se extienden desde los cuartiles Q1 y Q3 hasta los valores más extremos de la muestra que no se consideran outliers.

<a >
    <img src="https://drive.google.com/uc?export=view&id=1POubQHy2uK2RNjFMmHcu2xQpyKyzV_Eq" width="500" align="center">
</a>

In [ ]:
# Crea un diagrama de caja de las columnas "median_income" y "ocean_proximity" del conjunto de datos Housing_df
sns.catplot(x="median_income", y="ocean_proximity", kind="box", data=Housing_df, height = 5)

# Agrega un título al diagrama de caja
plt.title("Salario por proximidad al océano")

In [ ]:
sns.catplot(x="population", y="ocean_proximity", kind="box", data=Housing_df,  height = 5)
plt.title("Población por proximidad al océano")

In [ ]:
sns.catplot(x="median_income", y="ocean_proximity", kind="box", data=Housing_df,  height = 5)
plt.title("Ingreso medio por proximidad al océano")

### Visualización de datos geográficos

Hagamos un diagrama de dispersión de los valores de latitud y longitud para ver si podemos identificar algún patrón interesante

In [ ]:
# Crea una figura de Matplotlib con un tamaño de 6x8 pulgadas
figure = plt.figure(figsize=(6,8))

# Crea un diagrama de dispersión de las columnas "longitude" y "latitude" del conjunto de datos Housing_df
# con un tamaño de marcador de 15 y un color purpura
plt.scatter(Housing_df['longitude'], Housing_df['latitude'], s=15, c='purple')

# Muestra el diagrama de dispersión
plt.show()

Para que este gráfico sea aún más informativo, coloreemos los puntos según el valor medio de la vivienda; Usaremos el mapa de colores (paleta) de viridis ya que ha sido cuidadosamente diseñado para datos que tienen una naturaleza secuencial (es decir, valores bajos a altos):

In [ ]:
# Crea un diagrama de dispersión de los datos de longitud y latitud del conjunto de datos Housing_df
# con una transparencia de 0.1, una paleta de colores viridis y un tamaño de punto proporcional a la población
fig = sns.scatterplot(
    x="longitude",
    y="latitude",
    data=Housing_df,
    alpha=0.1,
    hue="median_house_value",
    palette="viridis",
    size=Housing_df["population"] / 100
)

# Coloca la leyenda del diagrama en el centro izquierdo de la figura
fig.legend(loc="center left", bbox_to_anchor=(1.01, 0.6), ncol=1);

<a>
    <img src="https://drive.google.com/uc?export=view&id=1wmACp1LBzVTeoG4dYiijy_B2EX_kdzTf" width="800" align="center">
</a>

**Fuente**: https://es.wikipedia.org/wiki/California

<h2 id="prep"> Paso 3: Preparación de Datos </h2>

### Limpieza de datos

Cuando recibe un nuevo conjunto de datos al comienzo de un proyecto, la primera tarea generalmente implica algún tipo de limpieza de datos.

Para resolver la tarea en cuestión, es posible que necesite datos de múltiples fuentes que deba combinar en una tabla unificada. Sin embargo, esto suele ser una tarea complicada; Las diferentes fuentes de datos pueden tener diferentes convenciones de nomenclatura, algunas de ellas pueden ser generadas por humanos, mientras que otras son informes automáticos del sistema. Una lista de cosas por las que normalmente tienes que pasar es la siguiente:

* Fusionar varias fuentes en una tabla
* Eliminar entradas duplicadas
* Limpiar entradas corruptas
* Manejar datos faltantes

Aunque construir algoritmos que sean capaces de clasificar datos o estimar resultados es posiblemente la parte más interesante de la ciencia de datos, la limpieza de datos es la que ocupa la mayor parte del tiempo. Según un estudio de  [CrowdFlower](https://www.forbes.com/sites/gilpress/2016/03/23/data-preparation-most-time-consuming-least-enjoyable-data-science-task-survey-says/?sh=705a6b5d6f63)., los científicos de datos dedican entre el 60% y el 80% de su tiempo a preparar conjuntos de datos para algoritmos de aprendizaje automático.

#### **Valores perdidos**
En general, los algoritmos de aprendizaje automático no funcionarán con los datos faltantes y, en general, tiene tres opciones para manejarlos:

* Deshazte de las filas correspondientes.
* Deshacerse de toda la característica o columna
* Reemplace los valores faltantes con algún valor como cero o la media, mediana de la columna.

In [ ]:
# Verificación de datos
print(len(Housing_df))
pd.DataFrame(Housing_df).isna().sum()

Veamos nuestra primera opción para manejar los datos faltantes: deshacernos de las filas. Para lograr esto podemos utilizar el método <code>DataFrame.dropna()</code>  de la siguiente manera:

In [ ]:
Housing_df = Housing_df.dropna()
Housing_df

#### **Ajustar Tipo de dato**

In [ ]:
Housing_df.info()

pandas tiene un tipo categórico especial para almacenar datos que utiliza la representación o codificación categórica basada en números enteros. Por ejemplo, <code>Housing_df['ocean_proximity']</code> es un pandas. Serie de objetos de cadena de Python <code>['NEAR BAY', '<1H OCEAN', 'INLAND', 'NEAR OCEAN', 'ISLAND ']</code>. Podemos convertir una columna <code>pandas.DataFrame</code> a categórica de la siguiente manera:

In [ ]:
Housing_df['ocean_proximity'] = Housing_df['ocean_proximity'].astype('category')

Housing_df.info()

#### Selección de Variables

Hay varias razones por las que se podría quitar una categoría de una variable a predecir. Algunas de las razones más comunes son:

* **La categoría es demasiado pequeña**. Si una categoría es muy pequeña, puede no tener suficiente información para entrenar un modelo de aprendizaje automático. Esto puede conducir a un modelo que no es preciso o que no es capaz de generalizar a nuevos datos.
* **La categoría es irrelevante para la predicción**. Si una categoría no está relacionada con la variable a predecir, no es necesario incluirla en el modelo. Esto puede ayudar a mejorar la precisión del modelo.
* **La categoría es problemática**. Si una categoría contiene datos erróneos o incompletos, puede ser mejor quitarla del modelo. Esto puede evitar que el modelo aprenda de datos incorrectos.

En este caso la categoria **ISLAN** de la variable **ocean_proximity** tienen solo 5 registros lo que para la creación de un modelo son muy pocos registros, dado que es muy probable que no quede bien entrenada esta categoria mejor la retiramos del proceso de modelado

In [ ]:
filtro = Housing_df['ocean_proximity'] != 'ISLAND'
Housing_df = Housing_df[filtro]

In [ ]:
Housing_df.head()

#### **Combinaciones de atributos**

En algún momento, las combinaciones de atributos son más significativas e interesantes en términos de resolver los problemas comerciales, por ejemplo,

* Habitaciones por hogar: el número total de habitaciones por distrito no es útil, pero las habitaciones por hogar pueden ser interesantes.
* relación dormitorio/habitación total
* población por hogar

In [ ]:
# calculated attributes
Housing_df['rooms_per_household'] = Housing_df['total_rooms']/Housing_df['households']
Housing_df['bedrooms_per_room'] = Housing_df['total_bedrooms']/Housing_df['total_rooms']
Housing_df['population_per_household'] = Housing_df['population']/Housing_df['households']

In [ ]:
# checkout the correlations again
corr_matrix = Housing_df.select_dtypes(include=[np.number]).corr()
corr_matrix['median_house_value'].sort_values(ascending=False)

Dos hallazgos después de combinar atributos:

* rooms_per_household está ligeramente más correlacionado (0,146285) con el valor de la casa que total_rooms (0,135097)
* bedrooms_per_room está mucho más correlacionado (-0,259984) que total_rooms (0,135097) y total_bedrooms (0,047689): las casas con una menor relación bedroom/room son más caras: esto tiene sentido, las casas más caras pueden tener más oficinas, estudios, salas de juegos, etc.

#### **Text and Categorial Attributes**

La mayoría de los algoritmos de ML funcionan mejor con números. Por lo tanto, a menudo necesitamos convertir atributos de texto en atributos numéricos. Para ocean_proximity, tenemos dos formas de manejar este problema:

* Asigne cada categoría a un número, como "<1H OCEAN" es 0, "INLAND" es 1, 'NEAR OCEAN' es 4, etc. El problema con esta solución es que el algoritmo ML puede pensar que 4 es mayor que 0, lo que podría causar un problema.
* Para solucionar el problema en 1, también podemos crear una variable binaria para cada atributo, lo que se denomina codificación one-hot (solo una es 1 hot, todas las demás son 0 cold)


##### **ONE-HOT Encoding**

La codificación **one-hot** es una técnica utilizada para representar datos categóricos como vectores numéricos. Esto es necesario para los algoritmos de aprendizaje automático, que normalmente requieren entradas numéricas. La codificación one-hot funciona creando una nueva variable binaria para cada categoría única en la variable categórica. El valor de cada variable binaria es 1 si la categoría correspondiente está presente y 0 en caso contrario.

In [ ]:
Housing_df.head(5)


<a>
    <img src="https://drive.google.com/uc?export=view&id=1o0W447I3pjcoKBs72RZ1Ne-rBHzNa7fW" width="700" align="center">
</a>

**Fuente**: https://datagy.io/sklearn-one-hot-encode/

Por lo tanto, un enfoque alternativo es aplicar una técnica conocida como codificación **one-hot**, donde creamos una característica binaria por categoría. En pandas podemos hacer esto simplemente ejecutando <code>pandas.get_dummies()</code>

In [ ]:
Housing_df = pd.get_dummies(Housing_df)
Housing_df.head()

<h2 id="mod"> Paso 4: Modelamiento </h2>

### **Creación de muestras para entrenamiento y validación**

La variable X se asigna a un subconjunto de Housing_df sin la columna median_house_value. Para ello, se utiliza el método <code>drop()</code> del DataFrame. El primer argumento de drop() es el nombre de la columna que queremos eliminar. El segundo argumento, <code>axis=1</code>, indica que queremos eliminar una columna, no una fila.

La variable y se asigna a un subconjunto de Housing_df que solo contiene la columna ,<code>median_house_value</code>.

In [ ]:
X = Housing_df.drop('median_house_value', axis=1)
y = Housing_df['median_house_value']

Una forma de medir qué tan bien se generalizará un modelo a casos nuevos es dividir los datos en dos conjuntos: **el conjunto de entrenamiento** y el **conjunto de validación**. Como lo implican estos nombres, usted entrena su modelo usando el conjunto de entrenamiento y lo valida usando el conjunto de validación. La tasa de error en el conjunto de validación es una medida que se utiliza como referencia para saber si el modelo se ajusta bien a datos con los cuales no fueron entrenados

In [ ]:
# Dividimos el conjunto de datos `X` y `y` en dos conjuntos: un conjunto de entrenamiento `X_train` y `y_train`,
# y un conjunto de validación `X_valid` y `y_valid`.
# El argumento `test_size` especifica el porcentaje de datos que queremos asignar al conjunto de validación.
# El argumento `random_state` se utiliza para garantizar que la división del conjunto de datos sea reproducible.
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Imprimimos el número de filas de los conjuntos de entrenamiento y validación.
# Esto nos permite comprobar que la división del conjunto de datos se ha realizado correctamente.
print(f'{len(X_train)} filas de entrenamiento + {len(X_valid)} filas de validación')

### **Selección del modelo**

Podemos utilizar esta función para entrenar un modelo de regresión aleatoria para predecir el precio de una casa.

Para ello, primero debemos crear un objeto <code>RandomForestRegressor</code>.
A continuación, podemos entrenar el modelo utilizando la función <code>fit()</code>.
La función <code>fit()</code> toma los argumentos <code>X_train y y_train</code>.
Una vez que el modelo haya sido entrenado, podemos utilizarlo para predecir el precio de una casa nueva.

El módulo <code>RandomForestRegressor</code> nos permite crear modelos de regresión aleatoria.
Un modelo de regresión aleatoria es un tipo de modelo de aprendizaje automático que utiliza un conjunto de árboles de decisión para hacer predicciones.

Los árboles de decisión son una forma de <code>aprendizaje automático supervisado</code> que se utiliza para clasificar o **predecir valores continuos**, funcionan dividiendo el espacio de datos en regiones, cada una de las cuales se asigna a una clase o valor.

Un modelo de regresión aleatoria funciona construyendo un conjunto de árboles de decisión de forma aleatoria, esto ayuda a reducir el riesgo de sobreajuste.

In [ ]:
# Importamos el módulo `RandomForestRegressor` de la librería `sklearn.ensemble`

from sklearn.ensemble import RandomForestRegressor


<a>
    <img src="https://drive.google.com/uc?export=view&id=17BhSGqKUKFFYzSTMf5-Gu4fJNs390Sov" width="600" align="center">
</a>


**Fuente**: https://www.analyticsvidhya.com/blog/2021/06/understanding-random-forest/

En el caso de que la variable a predecir sea **categórica** como es el caso de la imagen anterior se selecciona el resultado usando la clase con mayor votación en cada arbol, en caso de que sea **numérica** la variable a predecir, se obtiene el resultado a traves de un promedio entre todos los arboles.

In [ ]:
# El parámetro `n_estimators` especifica el número de árboles que queremos utilizar en el modelo.
# El parámetro `n_jobs` especifica el número de núcleos que queremos utilizar para entrenar el modelo.
# El parámetro `random_state` se utiliza para garantizar que la creación del modelo sea reproducible.

model = RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=42)

In [ ]:
# La función `fit()` toma los siguientes argumentos:
#
# * `X`: El conjunto de datos de entrada.
# * `y`: La variable objetivo.

# La función devuelve el modelo entrenado.

model.fit(X_train, y_train)

<h2 id="eva"> Paso 5: Evalucación </h2>

In [ ]:
# A continuación, podemos utilizar la función `predict()` para realizar predicciones sobre el conjunto de datos de validación.
# Las predicciones se pueden utilizar para evaluar el rendimiento del modelo.

y_pred = model.predict(X_valid)

In [ ]:
# Calculamos el error absoluto medio entre las predicciones y los valores reales del conjunto de validación.
# El error absoluto medio es una métrica de evaluación que mide la diferencia promedio entre las predicciones
# y los valores reales.

mean_absolute_error(y_valid, y_pred)

Para tener una idea de la frecuencia con la que nuestro modelo predice valores cercanos a los valores esperados, trazaremos las etiquetas **median_house_value** reales del conjunto de datos de prueba con el valor predicho generado por nuestro modelo final:

In [ ]:
def plot_prediction_error(fitted_model, X, y):
    """
     Una función de utilidad para visualizar los errores de predicción de los modelos de regresión.

     Argumentos:
         fitted_model: un modelo de regresión de aprendizaje científico.
         X: La matriz de características sobre la que generar predicciones.
         y: el vector objetivo compara las predicciones.
     """
    y_pred = model.predict(X)
    plt.figure(figsize=(8, 4))
    sns.scatterplot(x=y, y=y_pred)
    sns.lineplot(x=[y.min(), y.max()], y=[y.min(), y.max()], lw=2, color="r")
    plt.xlabel("Real: Median House Price")
    plt.ylabel("predicción: Median House Price")
    plt.title(f"Error de Predicción del modelo {model.__class__.__name__}")
    plt.show()

In [ ]:
plot_prediction_error(model, X_valid, y_valid)

Lo que buscamos aquí es una relación clara y lineal entre los valores previstos y reales. La **línea roja** denota lo que podría considerarse un modelo "óptimo", por lo que queremos que nuestros puntos se agrupen alrededor de esta línea. Podemos ver que, aparte de algunos valores atípicos, el bosque aleatorio funciona bastante bien. (De hecho, esos valores atípicos podrían sugerir que algo anda mal con los datos o que estas casas son especiales por razones que no se reflejan en los datos, lo que podria sugerir que encontrar mas variables para entrenar el modelo puede ser util)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

**La Raíz del Error Cuadrático Medio (RMSE)** es una métrica relacionada con el **Error Cuadrático Medio (ECM)** que se utiliza comúnmente para evaluar el rendimiento de modelos de regresión. La principal diferencia entre el ECM y el RMSE es que el RMSE toma la raíz cuadrada del ECM, lo que lo convierte en una medida en la misma escala que los valores reales. Esto facilita la interpretación, ya que el RMSE se expresa en las mismas unidades que la variable objetivo.

Supongamos que estamos construyendo un modelo de regresión para predecir las ventas mensuales de una tienda en función de variables como el gasto en publicidad, el precio de los productos, el tamaño de la tienda, etc. Después de entrenar el modelo y usarlo para hacer predicciones, calculamos el RMSE.


Si el RMSE es, por ejemplo, 1000, esto significa que, en promedio, nuestras predicciones tienen un error de 1000 unidades en términos de ventas mensuales. En este caso, el RMSE se expresa en la misma unidad que las ventas, lo que facilita la interpretación. Cuanto menor sea el RMSE, mejor será el rendimiento del modelo, ya que indicaría que las predicciones se acercan más a los valores reales de las ventas.


**El coeficiente de determinación R^2**  es una métrica comúnmente utilizada para evaluar el rendimiento de un modelo de regresión. R^2 mide la proporción de la variabilidad en la variable dependiente (variable objetivo) que es explicada por el modelo. Puede tomar valores entre 0 y 1, donde:

R^2 = 0: El modelo no explica ninguna variabilidad en los datos. Las predicciones son tan malas como simplemente usar la media de la variable dependiente.

R^2 = 1: El modelo explica toda la variabilidad en los datos. Las predicciones del modelo coinciden perfectamente con los valores reales.

In [ ]:
def rmse(y, yhat):
    """Una función de utilidad para calcular el error cuadrático medio (RMSE).

     Argumentos:
         y: valores reales para el objetivo.
         yhat : Las predicciones.

     Devoluciones:
         rmse: El RMSE.
    """
    return np.sqrt(mean_squared_error(y, yhat))

In [ ]:
def print_rf_scores(fitted_model):
    """Genera puntuaciones RMSE y R^2 a partir del modelo Random Forest."""

    yhat_train = fitted_model.predict(X_train)
    R2_train = fitted_model.score(X_train, y_train)
    yhat_valid = fitted_model.predict(X_valid)
    R2_valid = fitted_model.score(X_valid, y_valid)

    scores = {
        "RMSE en entrenamiento:": rmse(y_train, yhat_train),
        "R^2 en entrenamiento:": R2_train,
        "RMSE en validación:": rmse(y_valid, yhat_valid),
        "R^2 en validación:": R2_valid,
    }
    if hasattr(fitted_model, "oob_score_"):
        scores["OOB R^2:"] = fitted_model.oob_score_

    for score_name, score_value in scores.items():
        print(score_name, round(score_value, 3))

In [ ]:
print_rf_scores(model)

En la práctica, podemos obtener una visión global clasificando cada característica (variable independiente) en términos de su importancia para las predicciones del modelo. En scikit-learn, el modelo Random Forest tiene un atributo llamado <code>feature_importances_</code> que podemos usar para clasificar cada característica:

In [ ]:
def rf_feature_importance(fitted_model, df):
    '''
    La función `rf_feature_importance()` devuelve un DataFrame con las siguientes columnas:
    * `Column`: El nombre de la característica.
    * `Importance`: La importancia de la característica.

    La importancia de una característica se calcula como la contribución de la característica a la precisión del modelo.

    La función `sort_values()` ordena el DataFrame por la columna `Importance`, de mayor a menor.
    '''
    return pd.DataFrame(
        {"Column": df.columns, "Importance": fitted_model.feature_importances_}
    ).sort_values("Importance", ascending=False)

In [ ]:
feature_importance = rf_feature_importance(model, X)

# echa un vistazo a las 10 variables principales
feature_importance[:10]

En la tabla vemos que **median_income**, la **proximidad del océano EN TIERRA** y la **población por hogar** son las características más importantes; esto no es del todo sorprendente ya que el ingreso y la ubicación de la casa parecen ser buenos indicadores del valor de la misma. También podemos trazar la importancia de la característica para obtener una comprensión visual:

In [ ]:
def plot_feature_importance(feature_importance):
    return sns.barplot(y="Column", x="Importance", data=feature_importance, color='b')

plot_feature_importance(feature_importance);

<hr>

<div class="alert alert-block alert-info" style="margin-top: 20px">

<h2>Felicitaciones por llegar hasta aquí!</h2>

¡Has completado una nueva hazaña!

Has aprendido mucho sobre como llevar un proyecto de I.A.


<h3>¡Sigue aprendiendo y creciendo!</h3>
    
</div>

<hr>

<h2>Autores y colaborares:</h2>


<a href="https://www.linkedin.com/in/robinssondeantonio/" title="Robinsson Deantonio"> Robinsson S. Deantonio</a>

Basado en el trabajo de
<a href="https://lewtun.github.io/dslectures/" title="lewtun.github"> lewtun.github </a>


<hr>

<p>Copyright &copy; 2023. Este cuaderno y su código fuente se publican según los términos del <a href="https://cognitiveclass.ai/mit-license/">MIT License</a>.</p>